In [2]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)

SEED=42

#Pipeline
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [5]:
import re, string, unicodedata
# import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')

# Pipeline de entrenamiento:

In [6]:
from preprocessing import tokenize_text

def create_pipeline():
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize_text)),
        ('model', LogisticRegression(multi_class='multinomial', max_iter=1000))
    ])
    dump(pipeline, 'assets/pipeline.joblib')
    return pipeline

def train_evaluate_pipeline(pipeline, X_train, X_test, y_train, y_test):
    # Entrenar el pipeline
    pipeline.fit(X_train, y_train)
    dump(pipeline, 'assets/pipeline.joblib')

    # Predecir en los conjuntos de entrenamiento y prueba
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)

    # Calcular métricas de evaluación para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')

    # Calcular métricas de evaluación para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')

    # Calcular la matriz de confusión para el conjunto de prueba
    conf_matrix = confusion_matrix(y_test, y_test_pred)

    # Devolver las métricas de evaluación
    return {
        'Train Accuracy': train_accuracy,
        'Train Recall (Weighted)': train_recall,
        'Train F1-Score (Weighted)': train_f1,
        'Test Accuracy': test_accuracy,
        'Test Recall (Weighted)': test_recall,
        'Test F1-Score (Weighted)': test_f1,
        'Confusion Matrix': conf_matrix
    }

def get_top_features(pipeline):
    # Obtener el vectorizador TF-IDF del pipeline
    tfidf_vectorizer = pipeline.named_steps['tfidf']
    # Obtener los nombres de las características
    feature_names = tfidf_vectorizer.get_feature_names_out()
    # Obtener el modelo del pipeline
    model = pipeline.named_steps['model']
    coefficients = model.coef_

    # Inicializar la lista de palabras más importantes por score
    top_words_by_score = []

    # Obtener las palabras más influyentes para cada score
    for i, score_coefficients in enumerate(coefficients):
        sorted_indices = np.argsort(score_coefficients)
        top_indices = sorted_indices[-10:]  # Obtener los índices de las 10 palabras principales
        top_indices = top_indices[::-1]  # Invertir para obtener las palabras con los coeficientes más altos primero
        top_words = [feature_names[idx] for idx in top_indices]
        top_words_by_score.append(top_words)

    return top_words_by_score


#Cargar datos
df_original = pd.read_csv('../data/tipo1_entrenamiento_estudiantes.csv')
df_prep = df_original.drop_duplicates()
X_train, X_test, y_train, y_test = train_test_split(df_prep["Review"], df_prep["Class"], test_size=0.3, stratify=df_prep["Class"], random_state=42)

# Utilizar la función para evaluar el pipeline
pipeline = create_pipeline()
evaluation_metrics = train_evaluate_pipeline(pipeline, X_train, X_test, y_train, y_test)
top_features = get_top_features(pipeline)
for i, score_words in enumerate(top_features, start=1):
    print(f"Score {i}: {score_words}")

# Imprimir las métricas de evaluación
print("\nEvaluation Metrics:")
for metric, value in evaluation_metrics.items():
    print(f"{metric}: {value}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Score 1: ['pesim', 'peor', 'mal', 'suci', 'horribl', 'terribl', 'cobr', 'rob', 'habi', 'pag']
Score 2: ['mal', 'decepcion', 'habit', 'asign', 'pobr', 'nadi', 'esper', 'dolar', 'ped', 'oli']
Score 3: ['bastant', 'normal', 'embarg', 'aunqu', 'falt', 'monument', 'men', 'demasi', 'viv', 'general']
Score 4: ['buen', 'limpi', 'excelent', 'comod', 'agrad', 'disfrut', 'ciud', 'ques', 'centr', 'hermos']
Score 5: ['excelent', 'delici', 'recomend', 'increibl', 'encant', 'hermos', 'atencion', 'maravill', 'color', 'perfect']

Evaluation Metrics:
Train Accuracy: 0.7997070670084219
Train Recall (Weighted): 0.7997070670084219
Train F1-Score (Weighted): 0.7983510861142579
Test Accuracy: 0.4884713919726729
Test Recall (Weighted): 0.4884713919726729
Test F1-Score (Weighted): 0.4791736134413112
Confusion Matrix: [[ 78  86  38  15  20]
 [ 46 133 111  35  23]
 [ 11  57 162 152  84]
 [  4  24  81 254 226]
 [  1   4  31 149 517]]


In [5]:
def predict_with_pipeline(pipeline,input_df):
    # Realizar la predicción utilizando el pipeline
    predictions = pipeline.predict(input_df)

    # Devolver las predicciones
    return predictions

# Llamada a la función para obtener las predicciones
input_df = pd.read_csv('../data/tipo1_entrenamiento_estudiantes.csv')
# Seleccione 5 filas para predecir aleatoriamente
input_df_sample = input_df.sample(5)['Review']
print(input_df_sample)
# Cargar el pipeline desde el archivo joblib
pipeline = load('assets/pipeline.joblib')
predictions = predict_with_pipeline(pipeline,input_df_sample)

# Imprimir las predicciones
print("\nPredictions:")
for i, prediction in enumerate(predictions, start=1):
    print(f"Prediction {i}: {input_df_sample.iloc[i-1]} - Predicted Class: {prediction}")



600     Excelente lugar, muy atentos desde el Sr. Arma...
7358    .Visitamos la Habana hace algunos días con mis...
6827    El lugar es una maravilla que merece ser visit...
6299    El mejoramiento continuo debe ser importante ,...
7718    Excelente ubicación.. Buenos servicios alreded...
Name: Review, dtype: object

Predictions:
Prediction 1: Excelente lugar, muy atentos desde el Sr. Armando que nos llevo en el trasporte todo un profesional y bien puesta la camiseta de la hacienda y de Merida, el guía profesional 100% y el lugar se ve de ensueño para quedarse un par de noches,  felicidades - Predicted Class: 5
Prediction 2: .Visitamos la Habana hace algunos días con mis amigas y realmente  fue  muy desagradable pasar 2 días en este Hotel , está muy mal tenido , aire acondicionado en mal estado ,ducha mala , paredes con filtración de humedad... y que decir de los pasillos , la alfombra da susto... Al dar aviso a mantención reclamando el aire acondicionado , ya que era insufrible estar 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersio